In [ ]:
from neo4j import GraphDatabase

In [ ]:
url = ''
pwd = ''

driver = GraphDatabase.driver(uri=url, auth=("neo4j", pwd))

In [ ]:
query = 'MATCH (n) return COUNT(n)'

with driver.session() as session:
    result = session.run(query)
    print(result.single()['COUNT(n)'])

In [ ]:
import pandas as pd
import numpy as np
from py2neo import Graph, Node, Relationship

In [ ]:
graph = Graph(uri=url, auth=("neo4j", pwd))

In [ ]:
result = graph.query(query)
result

In [ ]:
type(result)

In [ ]:
print(result.data())

In [ ]:
print(result)

In [ ]:
name_query = "MATCH (p:Person {name: 'Eddard Stark'}) RETURN p.name as name"
result = graph.query(name_query)
for record in result:
    print(record['name'])

In [ ]:
query_2hops = "MATCH (p:Person {name: 'Eddard Stark'}) - [:INTERACTS_1*1..2] -> (p2) RETURN p2.name as name LIMIT 10"
result = graph.query(query_2hops)
for record in result:
    print(record['name'])

In [ ]:
battle_query = "MATCH (p1:Person) - [:DEFENDER_COMMANDER] -> (b:Battle) <- [:ATTACKER_COMMANDER] - (p2:Person) return p1.name as defender, b.name as battle, p2.name as attacker LIMIT 10"
battle_df = graph.query(battle_query).to_data_frame()
battle_df[:10]

In [ ]:
battle_array = graph.query(battle_query).to_ndarray()
battle_array , battle_array.dtype

In [ ]:
# a = Node('Person', name='Clair')
# b = Node('Person', name='Emil')
# ab = Relationship(a, 'KNOWS', b)
# graph.create(ab)
# graph.exists(a), graph.exists(ab), graph.exists(b)

In [ ]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [ ]:
conn = Neo4jConnection(url, 'neo4j', pwd)

In [ ]:
result = conn.query(battle_query)
print(result)

In [ ]:
result = conn.query('MATCH(n) return COUNT(n) as ct')
print(result[0]['ct'])

In [ ]:
starks_query = """
MATCH (p:Person {name: $name}) - [:INTERACTS_1] - (p2:Person) 
WHERE p2.name contains $fragment 
RETURN DISTINCT(p2.name) as name
"""
result = conn.query(starks_query, parameters={'name': 'Eddard Stark', 'fragment': 'Stark'})
for record in result:
    print(record['name'])

In [ ]:
query = "match (p1:Person) -[r]-> (p2:Person) return p1.name, TYPE(r) as the_type, p2.name ORDER BY the_type DESC LIMIT 100"
result = conn.query(query)
pd.DataFrame( dict(_) for _ in result)

In [ ]:
#in degrees
conn.query('match(p:Person) return p.name, size(()-[]->(p)) as inDegree order by inDegree desc LIMIT 1')

In [ ]:
#out degree
conn.query('match(p:Person) return p.name, size((p)-[]->()) as outDegree order by outDegree desc LIMIT 1')

In [ ]:
#shortest path
conn.query("match p=shortestpath( (:Person {name: 'Eddard Stark'}) -[*] -(:Person {name: 'Vance'})) return p")[0]

In [ ]:
#in memeory weighted graph
conn.query("call gds.graph.create.cypher('people-weighted', 'MATCH(n:Person) return id(n) as id', 'MATCH( (n:Person) - [r] -> (m:Person) ) return id(n) as source, id(m) as target, r.weight as weight')")

In [ ]:
property_query = """
call gds.graph.streamRelationshipProperty('people-weighted', 'weight')
YIELD sourceNodeId, targetNodeId, propertyValue as weight
WHERE TOSTRING(TOFLOAT(weight)) <> 'NaN'
RETURN
    gds.util.asNode(sourceNodeId).name as person1,
    gds.util.asNode(targetNodeId).name as person2,
    weight
ORDER BY weight DESC 
LIMIT 10
"""
conn.query(property_query)

In [ ]:
create_interactions_query = """
call gds.graph.create.cypher('interactions-weighted', 
'MATCH(n) WHERE n:Person or n:King return id(n) as id',
'MATCH( (n:Person) - [r: INTERACTS_1 | INTERACTS_5] -> (m:Person) ) return id(n) as source, id(m) as target, r.weight as weight')
"""
conn.query(create_interactions_query)

In [ ]:
list_em = """
call gds.graph.list()
"""
conn.query(list_em)

In [ ]:
drop_em = """
call gds.graph.drop('people-weighted')
"""
conn.query(drop_em)

drop_em = """
call gds.graph.drop('interactions-weighted')
"""
conn.query(drop_em)

In [ ]:
native_creation = """
call gds.graph.create('interactions', 'Person', 'INTERACTS')
"""

conn.query(native_creation)

In [ ]:
native_ucreation = """
call gds.graph.create('interactions-undirected', 'Person', {INTERACTS: {orientation: 'UNDIRECTED'}} )
"""

conn.query(native_ucreation)

In [ ]:
native_wcreation = """
call gds.graph.create('interactions-weighted', 'Person', {INTERACTS: {properties: 'weight'}} )
"""

conn.query(native_wcreation)

In [ ]:
native_belong = """
call gds.graph.create('people-houses', ['Person', 'House'], 'BELONGS_TO' )
"""

conn.query(native_belong)

In [ ]:
native_complicated = """
call gds.graph.create('complicate-graph', ['Person', 'House'], {INTERACTS: {properties: 'weight', orientation: 'UNDIRECTED'}, BELONGS_TO: {orientation: 'NATURAL'}} )
"""

conn.query(native_complicated)

In [ ]:
#pagerank

page_rank_call = """
CALL gds.pageRank.stream( "interactions-undirected", {} ) 
YIELD nodeId,score 
RETURN gds.util.asNode(nodeId).name as name, score 
ORDER by score DESC 
LIMIT 10;
"""

conn.query(page_rank_call)

In [ ]:
#weighted pagerank

weighted_page_rank_call = """
CALL gds.pageRank.stream( "interactions-weighted", {} ) 
YIELD nodeId,score 
RETURN gds.util.asNode(nodeId).name as name, score 
ORDER by score DESC 
LIMIT 10;
"""

conn.query(weighted_page_rank_call)

In [ ]:
#betweeness centrality (no weights supported)

betweeness_call = """
CALL gds.betweenness.stream( "interactions-undirected", {} ) 
YIELD nodeId,score 
RETURN gds.util.asNode(nodeId).name as name, score 
ORDER by score DESC 
LIMIT 10;
"""

conn.query(betweeness_call)

In [ ]:
#degree centrality (no weights supported)

degreeness_call = """
CALL gds.degree.stream( "interactions-weighted", {} ) 
YIELD nodeId,score 
RETURN gds.util.asNode(nodeId).name as name, score 
ORDER by score DESC 
LIMIT 10;
"""

conn.query(degreeness_call)

In [ ]:
#unweighted louvain

louvain_method = """
CALL gds.louvain.stream( "interactions-undirected", {} ) 
YIELD nodeId,communityId 
RETURN communityId, count(distinct nodeId) as members
ORDER by members DESC 
LIMIT 10;
"""

conn.query(louvain_method)

In [ ]:
#weighted louvain

weighted_louvain_method = """
CALL gds.louvain.stream( "interactions-weighted", {relationshipWeightProperty: "weight"} ) 
YIELD nodeId,communityId 
RETURN communityId, count(distinct nodeId) as members
ORDER by members DESC 
LIMIT 10;
"""

conn.query(weighted_louvain_method)

In [ ]:
#unweighted louvain modularity

write_louvain_method = """
CALL gds.louvain.write( "interactions-undirected", {writeProperty: "louv_community"} ) 
YIELD communityCount,modularity, modularities 
"""

conn.query(write_louvain_method)

In [ ]:
#weighted louvain modularity

write_weighted_louvain_method = """
CALL gds.louvain.write( "interactions-weighted", {relationshipWeightProperty: "weight", writeProperty: "louv_community_weighted"} ) 
YIELD communityCount,modularity, modularities 
"""

conn.query(write_weighted_louvain_method)

In [ ]:
# drop_em = """
# call gds.graph.drop('interactions-undirected"')
# """
# conn.query(drop_em)

# drop_em = """
# call gds.graph.drop('interactions-weighted')
# """
# conn.query(drop_em)